### Запуск QtLab

In [1]:
%run D:\qtlab_replacement\init.py
import matplotlib.pyplot as plt
#import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *

Failed loading module Labbrick:  [WinError 126] The specified module could not be found
Failed loading module Signal_Hound_SA:  [WinError 126] The specified module could not be found


In [2]:
from resonator_tools import circuit

In [3]:
rf_switch = nn_rf_switch(name='rf_switch', address='10.20.61.224')

In [4]:
import imp
imp.reload(sweep)
from scripts import data_reduce

# Спектроскопия

In [5]:
current_src = Yokogawa_GS210(address='GPIB::2::INSTR')
pna = Agilent_N5242A('pna', address = 'PNA')
#lo1 = Agilent_E8257D('lo1', address = 'PSG1')
#sa = Signal_Hound_SA('sa', serial = 61660066)
sa = Agilent_N9030A('sa', address='PXA')

In [6]:
lo1 = Agilent_E8257D('lo1', address = 'PSG1')

In [7]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")

True

In [8]:
pna.set_xlim(4e9, 8.5e9)
diap = np.linspace(-0.0015,0.0015, 301)
current_src.set_status(1)
current_src.set_range(0.1)
pna.set_power(-45)
pna.set_nop(4501)
pna.set_bandwidth(2000)
sweep.sweep(pna,(diap,current_src.set_current,"Current", "A"), filename="paramp dispersion")

Started at:  Jan 17 2018 12:22:57

Elapsed time: 0 h 14 m 44.89 s


{'S-parameter': (('Current', 'Frequency'),
  (array([ -1.50000000e-03,  -1.49000000e-03,  -1.48000000e-03,
           -1.47000000e-03,  -1.46000000e-03,  -1.45000000e-03,
           -1.44000000e-03,  -1.43000000e-03,  -1.42000000e-03,
           -1.41000000e-03,  -1.40000000e-03,  -1.39000000e-03,
           -1.38000000e-03,  -1.37000000e-03,  -1.36000000e-03,
           -1.35000000e-03,  -1.34000000e-03,  -1.33000000e-03,
           -1.32000000e-03,  -1.31000000e-03,  -1.30000000e-03,
           -1.29000000e-03,  -1.28000000e-03,  -1.27000000e-03,
           -1.26000000e-03,  -1.25000000e-03,  -1.24000000e-03,
           -1.23000000e-03,  -1.22000000e-03,  -1.21000000e-03,
           -1.20000000e-03,  -1.19000000e-03,  -1.18000000e-03,
           -1.17000000e-03,  -1.16000000e-03,  -1.15000000e-03,
           -1.14000000e-03,  -1.13000000e-03,  -1.12000000e-03,
           -1.11000000e-03,  -1.10000000e-03,  -1.09000000e-03,
           -1.08000000e-03,  -1.07000000e-03,  -1.06000000e-0

In [14]:
current_src.set_current(7.4e-3)

True

In [19]:
lo1.set_status(1)
lo1.set_frequency(4.70805e9)
power=np.linspace(-20,-10,21)
pna.set_xlim(4.6e9,4.62e9)
sweep.sweep(pna,(power,lo1.set_power,"Pump Power","dBm"),filename="Paramp pupm power sweep")

Started at:  Nov 16 2017 15:48:19

Elapsed time: 0 h 0 m 37.85 s


{'S-parameter': (('Pump Power', 'Frequency'),
  (array([-20. , -19.5, -19. , -18.5, -18. , -17.5, -17. , -16.5, -16. ,
          -15.5, -15. , -14.5, -14. , -13.5, -13. , -12.5, -12. , -11.5,
          -11. , -10.5, -10. ]),
   array([  4.60000000e+09,   4.60006667e+09,   4.60013333e+09,
            4.60020000e+09,   4.60026667e+09,   4.60033333e+09,
            4.60040000e+09,   4.60046667e+09,   4.60053333e+09,
            4.60060000e+09,   4.60066667e+09,   4.60073333e+09,
            4.60080000e+09,   4.60086667e+09,   4.60093333e+09,
            4.60100000e+09,   4.60106667e+09,   4.60113333e+09,
            4.60120000e+09,   4.60126667e+09,   4.60133333e+09,
            4.60140000e+09,   4.60146667e+09,   4.60153333e+09,
            4.60160000e+09,   4.60166667e+09,   4.60173333e+09,
            4.60180000e+09,   4.60186667e+09,   4.60193333e+09,
            4.60200000e+09,   4.60206667e+09,   4.60213333e+09,
            4.60220000e+09,   4.60226667e+09,   4.60233333e+09,
       

## Резонаторная спектроскопия

In [7]:
pna.set_timeout(30000)
pna.set_trigger_source("MAN")
pna.init()
pna.ask("*OPC?")
pna.get_tracedata()
pna.set_trigger_source("IMM")

res_data = pna.get_tracedata()
res_freqs = pna.get_freqpoints()
plt.figure("S21_amp")
plt.plot(res_freqs, res_data[0])
plt.figure("S21_ph")
plt.plot(res_freqs, res_data[1])


In [35]:
plt.figure("S21_amp")
plt.plot(res_freqs, 20.*np.log10(res_data[0]))
plt.figure("S21_ph")
plt.plot(res_freqs, res_data[1])

In [11]:
save_pkl({'type':"gain",}, {"Amp": res_data[0] , "Ph": res_data[1], "freqs": res_freqs } ,plot=False)

In [29]:
pna.set_xlim(6.58e9, 6.59e9)
pna.set_bandwidth(1e1)
pna.set_nop(201)
pna.set_power(-60)

True

In [10]:
pna.set_xlim(7.3e9,9.5e9)
pna.set_nop(600)

True

In [11]:
currents = np.linspace(-10e-3, 10e-3, 400)
current_src.set_voltage_compliance(20)
current_src.set_range(10e-3)
current_src.set_status(1)

True

In [28]:
powers = np.arange(-90,-55,1.)
sweep.sweep(pna, (powers, pna.set_power, 'PNA power'), filename='Gain')

Started at:  Oct 10 2017 20:07:34


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: PNA power:    -56,	average cycle time:    6s			
Elapsed time: 0 h 3 m 29.33 s


{'S-parameter': (('PNA power', 'Frequency'),
  (array([-90., -89., -88., -87., -86., -85., -84., -83., -82., -81., -80.,
          -79., -78., -77., -76., -75., -74., -73., -72., -71., -70., -69.,
          -68., -67., -66., -65., -64., -63., -62., -61., -60., -59., -58.,
          -57., -56.]),
   array([  7.84918758e+09,   7.84935452e+09,   7.84952147e+09,
            7.84968841e+09,   7.84985536e+09,   7.85002230e+09,
            7.85018925e+09,   7.85035619e+09,   7.85052314e+09,
            7.85069008e+09,   7.85085702e+09,   7.85102397e+09,
            7.85119091e+09,   7.85135786e+09,   7.85152480e+09,
            7.85169175e+09,   7.85185869e+09,   7.85202564e+09,
            7.85219258e+09,   7.85235953e+09,   7.85252647e+09,
            7.85269342e+09,   7.85286036e+09,   7.85302731e+09,
            7.85319425e+09,   7.85336120e+09,   7.85352814e+09,
            7.85369509e+09,   7.85386203e+09,   7.85402898e+09,
            7.85419592e+09,   7.85436287e+09,   7.85452981e+09,

In [10]:
lo1.set_frequency(8.74e9)
lo1.set_status(1)

True

In [ ]:
ex_powers = np.arange(8,18,0.01)
pna.set_centerfreq(4.37e9)
pna.set_span(2000e6)
pna.set_nop(201)
pna.set_bandwidth(100)
pna.set_power(-60)
sweep.sweep(pna, (ex_powers, lo1.set_power, 'excitation power'), filename='WTJPA Ic6.9uAC3.6pF ex8.74GHz')

In [7]:
lo1.set_status(0)

True

# Измерение усиления и шумов

In [6]:
# Мощность шума

In [109]:
from scipy.constants import Boltzmann, Planck
import time
sw_ch = 1
T1_sw = 4
T2_sw = 2
T1_Ts = [0.04]
T1_gains = [1]
T2_Ts = [0.04, 3.38]
T2_gains = [0.9, 0.1]
bw = 3e3



In [67]:
planck_function(8.6e9, T1_Ts, T1_gains)*3e3

8.5481948676574858e-21

In [18]:
pump_powers = np.linspace(-7, -4, 301)
probe_freq_start = pna.get_startfreq()#8.6189e9
probe_freq_end = pna.get_stopfreq()#['S-paramater']8.6689e9
rbw = 1e5
vbw = 100

sa.set_xlim(probe_freq_start, probe_freq_end)
sa.set_detector('rms')
sa.set_res_bw(rbw)
sa.set_video_bw(vbw)
#sa.set_ref(-20)
#current_src.set_current(62.6e-6)

True

In [5]:
m = sa.measure()
plt.plot(sa.get_freqpoints(), m['Power'])

In [4]:
m1=m

In [13]:
plt.plot(sa.get_freqpoints(), m['Power'])
plt.plot(sa.get_freqpoints(),m1["Power"])

In [12]:
save_pkl({"type":"noize"}, {"Cold":m1, "Hot":m, "F":sa.get_freqpoints(), "comment":"hot - 3.5K through 6 dB"},plot=False)

In [24]:
sa.set_xlim(6.5e9, 8e9)

In [30]:
time.sleep(240)
pna.set_status(False)
lo1.set_status(True)
m_cold_on = sweep.sweep(sa, ([0], lambda x: 0, 'Dummy'), filename='Paramp cold off terminated noise')#sa.measure()
lo1.set_status(False)
#plt.plot(sa.get_freqpoints(), m['Power'])
m_cold_off = sweep.sweep(sa, ([0], lambda x: 0, 'Dummy'), filename='Paramp cold on terminated noise')#sa.measure()
lo1.set_status(True)
#plt.plot(sa.get_freqpoints(), m['Power'])
pna.set_status(False)

Started at:  Jan 17 2018 19:20:01

Elapsed time: 0 h 3 m 55.06 s


Started at:  Jan 17 2018 19:23:56

Elapsed time: 0 h 3 m 55.2 s


True

In [31]:
time.sleep(240)
pna.set_status(False)
lo1.set_status(True)
m_hot_on = sweep.sweep(sa, ([0], lambda x: 0, 'Dummy'), filename='Paramp hot off terminated noise')#sa.measure()
lo1.set_status(False)
#plt.plot(sa.get_freqpoints(), m['Power'])
m_hot_off = sweep.sweep(sa, ([0], lambda x: 0, 'Dummy'), filename='Paramp hot on terminated noise')#sa.measure()
lo1.set_status(True)
#plt.plot(sa.get_freqpoints(), m['Power'])
pna.set_status(False)

Started at:  Jan 17 2018 19:38:58

Elapsed time: 0 h 3 m 54.9 s


Started at:  Jan 17 2018 19:42:54

Elapsed time: 0 h 3 m 54.91 s


True

In [59]:
sa.measure()

{'Power': array([-81.37501526, -81.62197113, -81.65299225, ..., -83.43331146,
        -83.43353271, -83.41014099])}

In [33]:
p = paramp(pna, lo1, current_src);

In [43]:
Gain, Noise = p.gain_noise(m_cold_on['Power'][1][0], [10**(m_cold_on['Power'][2]/10), 10**(m_hot_on['Power'][2]/10)], [[27e-3, 1.0]], [[3.387, 0.1], [27e-3, 0.9]], 1e6)

In [58]:
probe_powers = np.linspace(-70, -35, 36);
nop = 1501
pna.set_nop(nop)
pna.set_bandwidth(100)
time.sleep(240)
sweep.sweep(pna, (probe_powers, pna.set_power, 'Probe signal strength'), filename='Paramp (WTJPA) saturation')

Started at:  Jan 17 2018 20:21:36

Elapsed time: 0 h 8 m 29.04 s


{'S-parameter': (('Probe signal strength', 'Frequency'),
  (array([-70., -69., -68., -67., -66., -65., -64., -63., -62., -61., -60.,
          -59., -58., -57., -56., -55., -54., -53., -52., -51., -50., -49.,
          -48., -47., -46., -45., -44., -43., -42., -41., -40., -39., -38.,
          -37., -36., -35.]),
   array([  6.50000000e+09,   6.50100000e+09,   6.50200000e+09, ...,
            7.99800000e+09,   7.99900000e+09,   8.00000000e+09])),
  array([[-0.04686300+0.02887802j, -0.03115846+0.04496023j,
          -0.01973630+0.05004913j, ..., -0.04084054-0.00918119j,
          -0.04100863+0.00498566j, -0.04702640+0.01599258j],
         [-0.04465297+0.03138864j, -0.03196773+0.03647516j,
          -0.01609762+0.04998083j, ..., -0.04568706-0.00502704j,
          -0.05727162+0.00874494j, -0.03469763+0.01050992j],
         [-0.03793304+0.02805062j, -0.02649711+0.04497258j,
          -0.01273276+0.04524743j, ..., -0.04268286-0.0102803j ,
          -0.03786205+0.00089701j, -0.03860448+0.012

In [39]:
plt.figure()
plt.plot(m_cold_on['Power'][1][0], m_cold_on['Power'][2], label='cold')
plt.plot(m_cold_on['Power'][1][0], m_hot_on['Power'][2], label='hot')
plt.legend()

In [54]:
from scipy.constants import h, k
plt.figure()
plt.plot(m_cold_on['Power'][1][0]/1e9, Noise)
plt.ylim([0, 5])
plt.plot(m_cold_on['Power'][1][0]/1e9, m_cold_on['Power'][1][0]*h/k*0.5, linestyle=':')
plt.xlim([6.6, 8])
plt.grid()
plt.xlabel('Frequency, GHz')
plt.ylabel('Noise temperature, K')

In [57]:
from scipy.constants import h, k
plt.figure()
plt.plot(m_cold_on['Power'][1][0]/1e9, np.log10(Gain)*10)
#plt.ylim([0, 5])
#plt.plot(m_cold_on['Power'][1][0]/1e9, m_cold_on['Power'][1][0]*h/k*0.5, linestyle=':')
plt.xlim([6.6, 8])
plt.grid()
plt.xlabel('Frequency, GHz')
plt.ylabel('Full system gain, dB')

c:\python36\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in log10
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
save_pkl({"type":"noize"}, {"Cold":m_cold_off, "Hot":m_cold_on, "F":sa.get_freqpoints(), "comment":"hot - 3.5K through 6 dB"},plot=False)

In [11]:
m

{'Power': array([-85.54449463, -86.03117371, -84.81076813, ..., -84.65739441,
        -85.26682281, -84.71646881])}

In [103]:
plt.plot(m['Power'])

In [10]:
pump_freq = lo1.get_frequency()
lo1.set_status(False)

noises = []

pna.set_power(-95)
#rf_switch.do_set_switch(1, channel=sw_ch)
#time.sleep(120)
noises.append(sweep.sweep(sa, (pump_powers[0:1], lo1.set_power, 'Excitation power'), \
                        filename='WTJPA Ic6.9uA res pump{0:2.4f}GHz switch 1'.format(pump_freq/1e9)))
lo1.set_status(True)

#rf_switch.do_set_switch(1, channel=sw_ch)
#print (rf_switch.do_get_switch(channel=sw_ch))
#time.sleep(600)
noises.append(sweep.sweep(sa, (pump_powers, lo1.set_power, 'Excitation power'), \
                        filename='WTJPA Ic6.9uA res pump{0:2.4f}GHz switch 1'.format(pump_freq/1e9)))

rf_switch.do_set_switch(2, channel=sw_ch)
print (rf_switch.do_get_switch(channel=sw_ch))
time.sleep(600)
noises.append(sweep.sweep(sa, (pump_powers, lo1.set_power, 'Excitation power'), \
                        filename='WTJPA Ic6.9uA res pump{0:2.4f}GHz switch 2'.format(pump_freq/1e9)))

rf_switch.do_set_switch(4, channel=sw_ch)
print (rf_switch.do_get_switch(channel=sw_ch))
time.sleep(600)
noises.append(sweep.sweep(sa, (pump_powers, lo1.set_power, 'Excitation power'), \
                        filename='WTJPA Ic6.9uA res pump{0:2.4f}GHz switch 4'.format(pump_freq/1e9)))

#rf_switch.do_set_switch(5, channel=sw_ch)
#print (rf_switch.do_get_switch(channel=sw_ch))
#time.sleep(600)
#noises.append(sweep.sweep(sa, (pump_powers, lo1.set_power, 'Excitation power'), \
#                        filename='WTJPA Ic6.9uA res pump{0:2.4f}GHz switch 5'.format(pump_freq/1e9)))

Started at:  Jun 14 2017 20:41:29
First sweep...

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Excitation power:     -6,	average cycle time:   43s	
Elapsed time: 0 h 0 m 43.09 s
Started at:  Jun 14 2017 20:42:16
Time left: 0 h 0 m 0.0 s,	parameter values: Excitation power:     -3,	average cycle time:   40s	s		
Elapsed time: 3 h 18 m 29.34 s
0
Started at:  Jun 15 2017 00:11:21
Time left: 0 h 0 m 0.0 s,	parameter values: Excitation power:     -3,	average cycle time:   40s	s		
Elapsed time: 3 h 18 m 34.75 s
0
Started at:  Jun 15 2017 03:40:21
Time left: 0 h 0 m 0.0 s,	parameter values: Excitation power:     -3,	average cycle time:   40s	s		
Elapsed time: 3 h 18 m 34.02 s
0
Started at:  Jun 15 2017 07:09:19
Time left: 0 h 0 m 0.0 s,	parameter values: Excitation power:     -3,	average cycle time:   40s	s		
Elapsed time: 3 h 18 m 43.55 s


In [160]:
from scipy.signal import gaussian, convolve
gains = np.zeros_like(noises[1]['Power'][2])
noise_Ts = np.zeros_like(noises[1]['Power'][2])
filt = gaussian(300, 50)
filt = filt/np.sum(filt)
for power_id, pump_power in enumerate(pump_powers):
    gain, noise_T = gain_noise(noises[1]['Power'][1][1], [convolve(n['Power'][2][power_id,:], filt, mode='same') for n in [noises[3], noises[2]]])
    gains[power_id, :] = gain
    noise_Ts[power_id, :] = noise_T

[[  1.00000000e+00   8.56696843e-21]
 [  1.00000000e+00   2.17275158e-20]] [  1.07239815e-13   1.11955825e-13] [  1.04169888e-13   3.58344557e+05]
[[  1.00000000e+00   8.56696890e-21]
 [  1.00000000e+00   2.17275162e-20]] [  1.08949934e-13   1.13749911e-13] [  1.05825349e-13   3.64724695e+05]
[[  1.00000000e+00   8.56696937e-21]
 [  1.00000000e+00   2.17275166e-20]] [  1.10659498e-13   1.15543607e-13] [  1.07480146e-13   3.71117487e+05]
[[  1.00000000e+00   8.56696984e-21]
 [  1.00000000e+00   2.17275170e-20]] [  1.12367693e-13   1.17335751e-13] [  1.09133694e-13   3.77496310e+05]
[[  1.00000000e+00   8.56697031e-21]
 [  1.00000000e+00   2.17275175e-20]] [  1.14073416e-13   1.19125524e-13] [  1.10784703e-13   3.83882850e+05]
[[  1.00000000e+00   8.56697079e-21]
 [  1.00000000e+00   2.17275179e-20]] [  1.15775705e-13   1.20912702e-13] [  1.12431734e-13   3.90333049e+05]
[[  1.00000000e+00   8.56697126e-21]
 [  1.00000000e+00   2.17275183e-20]] [  1.17474126e-13   1.22697224e-13] [  1.14

[[  1.00000000e+00   8.56697220e-21]
 [  1.00000000e+00   2.17275192e-20]] [  1.20478375e-13   1.28300234e-13] [  1.15386668e-13   5.94341465e+05]
[[  1.00000000e+00   8.56697267e-21]
 [  1.00000000e+00   2.17275196e-20]] [  1.22186856e-13   1.30084069e-13] [  1.17046097e-13   6.00067196e+05]
[[  1.00000000e+00   8.56696843e-21]
 [  1.00000000e+00   2.17275158e-20]] [  1.07861076e-13   1.13348591e-13] [  1.04288933e-13   4.16967051e+05]
[[  1.00000000e+00   8.56696890e-21]
 [  1.00000000e+00   2.17275162e-20]] [  1.09569690e-13   1.15194078e-13] [  1.05908447e-13   4.27367341e+05]
[[  1.00000000e+00   8.56696937e-21]
 [  1.00000000e+00   2.17275166e-20]] [  1.11277695e-13   1.17040229e-13] [  1.07526524e-13   4.37864354e+05]
[[  1.00000000e+00   8.56696984e-21]
 [  1.00000000e+00   2.17275170e-20]] [  1.12984913e-13   1.18886226e-13] [  1.09143403e-13   4.48409419e+05]
[[  1.00000000e+00   8.56697031e-21]
 [  1.00000000e+00   2.17275175e-20]] [  1.14690662e-13   1.20730978e-13] [  1.10

 [  1.00000000e+00   2.17275187e-20]] [  1.17697812e-13   1.23573039e-13] [  1.13873282e-13   4.46427303e+05]
[[  1.00000000e+00   8.56697220e-21]
 [  1.00000000e+00   2.17275192e-20]] [  1.19380129e-13   1.25336892e-13] [  1.15502523e-13   4.52622760e+05]
[[  1.00000000e+00   8.56697267e-21]
 [  1.00000000e+00   2.17275196e-20]] [  1.21057325e-13   1.27094685e-13] [  1.17127252e-13   4.58746962e+05]
[[  1.00000000e+00   8.56696843e-21]
 [  1.00000000e+00   2.17275158e-20]] [  1.05296724e-13   1.12800418e-13] [  1.00412132e-13   5.70165789e+05]
[[  1.00000000e+00   8.56696890e-21]
 [  1.00000000e+00   2.17275162e-20]] [  1.06984338e-13   1.14583903e-13] [  1.02037338e-13   5.77450493e+05]
[[  1.00000000e+00   8.56696937e-21]
 [  1.00000000e+00   2.17275166e-20]] [  1.08671995e-13   1.16365278e-13] [  1.03663988e-13   5.84571663e+05]
[[  1.00000000e+00   8.56696984e-21]
 [  1.00000000e+00   2.17275170e-20]] [  1.10359316e-13   1.18144152e-13] [  1.05291711e-13   5.91528309e+05]
[[  1.00

 [  1.00000000e+00   2.17275196e-20]] [  1.21838477e-13   1.32920548e-13] [  1.14624506e-13   8.42067647e+05]
[[  1.00000000e+00   8.56696843e-21]
 [  1.00000000e+00   2.17275158e-20]] [  1.07147698e-13   1.16289149e-13] [  1.01196994e-13   6.94610256e+05]
[[  1.00000000e+00   8.56696890e-21]
 [  1.00000000e+00   2.17275162e-20]] [  1.08852256e-13   1.18133317e-13] [  1.02810671e-13   7.05218454e+05]
[[  1.00000000e+00   8.56696937e-21]
 [  1.00000000e+00   2.17275166e-20]] [  1.10556257e-13   1.19976751e-13] [  1.04423906e-13   7.15813291e+05]
[[  1.00000000e+00   8.56696984e-21]
 [  1.00000000e+00   2.17275170e-20]] [  1.12259189e-13   1.21818513e-13] [  1.06036466e-13   7.26362208e+05]
[[  1.00000000e+00   8.56697031e-21]
 [  1.00000000e+00   2.17275175e-20]] [  1.13960490e-13   1.23657528e-13] [  1.07648120e-13   7.36826363e+05]
[[  1.00000000e+00   8.56697079e-21]
 [  1.00000000e+00   2.17275179e-20]] [  1.15659491e-13   1.25492740e-13] [  1.09258453e-13   7.47176292e+05]
[[  1.00

 [  1.00000000e+00   2.17275187e-20]] [  1.15970075e-13   1.28818992e-13] [  1.07605962e-13   9.76320832e+05]
[[  1.00000000e+00   8.56697220e-21]
 [  1.00000000e+00   2.17275192e-20]] [  1.17629929e-13   1.30649155e-13] [  1.09154951e-13   9.89261795e+05]
[[  1.00000000e+00   8.56697267e-21]
 [  1.00000000e+00   2.17275196e-20]] [  1.19284362e-13   1.32473152e-13] [  1.10699004e-13   1.00214608e+06]
[[  1.00000000e+00   8.56696843e-21]
 [  1.00000000e+00   2.17275158e-20]] [  1.02960481e-13   1.14810530e-13] [  9.52465922e-14   9.00422202e+05]
[[  1.00000000e+00   8.56696890e-21]
 [  1.00000000e+00   2.17275162e-20]] [  1.04626277e-13   1.16647671e-13] [  9.68008489e-14   9.13441844e+05]
[[  1.00000000e+00   8.56696937e-21]
 [  1.00000000e+00   2.17275166e-20]] [  1.06292039e-13   1.18483466e-13] [  9.83559265e-14   9.26361710e+05]
[[  1.00000000e+00   8.56696984e-21]
 [  1.00000000e+00   2.17275170e-20]] [  1.07956793e-13   1.20317496e-13] [  9.99104881e-14   9.39224114e+05]
[[  1.00

...

 [  1.00000000e+00   2.17275192e-20]] [  1.49378326e-13   1.66755724e-13] [  1.38066358e-13   1.32041610e+06]
[[  1.00000000e+00   8.56697267e-21]
 [  1.00000000e+00   2.17275196e-20]] [  1.51483183e-13   1.69066532e-13] [  1.40037149e-13   1.33606524e+06]
[[  1.00000000e+00   8.56696843e-21]
 [  1.00000000e+00   2.17275158e-20]] [  1.34280903e-13   1.47035543e-13] [  1.25978163e-13   9.69157284e+05]
[[  1.00000000e+00   8.56696890e-21]
 [  1.00000000e+00   2.17275162e-20]] [  1.36413084e-13   1.49403319e-13] [  1.27956982e-13   9.87058838e+05]
[[  1.00000000e+00   8.56696937e-21]
 [  1.00000000e+00   2.17275166e-20]] [  1.38544375e-13   1.51770186e-13] [  1.29934922e-13   1.00495903e+06]
[[  1.00000000e+00   8.56696984e-21]
 [  1.00000000e+00   2.17275170e-20]] [  1.40674152e-13   1.54135577e-13] [  1.31911322e-13   1.02286215e+06]
[[  1.00000000e+00   8.56697031e-21]
 [  1.00000000e+00   2.17275175e-20]] [  1.42801247e-13   1.56499024e-13] [  1.33884562e-13   1.04082128e+06]
[[  1.00

 [  1.00000000e+00   2.17275187e-20]] [  1.47217299e-13   1.61807587e-13] [  1.37719625e-13   1.10863839e+06]
[[  1.00000000e+00   8.56697220e-21]
 [  1.00000000e+00   2.17275192e-20]] [  1.49298008e-13   1.64093210e-13] [  1.39666943e-13   1.12420877e+06]
[[  1.00000000e+00   8.56697267e-21]
 [  1.00000000e+00   2.17275196e-20]] [  1.51372316e-13   1.66370783e-13] [  1.41608933e-13   1.13965378e+06]
[[  1.00000000e+00   8.56696843e-21]
 [  1.00000000e+00   2.17275158e-20]] [  1.25426958e-13   1.46294611e-13] [  1.11842984e-13   1.58562200e+06]
[[  1.00000000e+00   8.56696890e-21]
 [  1.00000000e+00   2.17275162e-20]] [  1.27464419e-13   1.48601341e-13] [  1.13705162e-13   1.60608230e+06]
[[  1.00000000e+00   8.56696937e-21]
 [  1.00000000e+00   2.17275166e-20]] [  1.29502769e-13   1.50906817e-13] [  1.15569624e-13   1.62637977e+06]
[[  1.00000000e+00   8.56696984e-21]
 [  1.00000000e+00   2.17275170e-20]] [  1.31540626e-13   1.53210169e-13] [  1.17434653e-13   1.64655332e+06]
[[  1.00

 [  1.00000000e+00   2.17275196e-20]] [  1.51980606e-13   1.59676239e-13] [  1.46971066e-13   5.84750270e+05]
[[  1.00000000e+00   8.56696843e-21]
 [  1.00000000e+00   2.17275158e-20]] [  1.30102739e-13   1.41743924e-13] [  1.22524811e-13   8.84551782e+05]
[[  1.00000000e+00   8.56696890e-21]
 [  1.00000000e+00   2.17275162e-20]] [  1.32194156e-13   1.43982411e-13] [  1.24520492e-13   8.95726844e+05]
[[  1.00000000e+00   8.56696937e-21]
 [  1.00000000e+00   2.17275166e-20]] [  1.34285591e-13   1.46219956e-13] [  1.26516815e-13   9.06828953e+05]
[[  1.00000000e+00   8.56696984e-21]
 [  1.00000000e+00   2.17275170e-20]] [  1.36375738e-13   1.48455903e-13] [  1.28512053e-13   9.17907449e+05]
[[  1.00000000e+00   8.56697031e-21]
 [  1.00000000e+00   2.17275175e-20]] [  1.38463272e-13   1.50689323e-13] [  1.30504620e-13   9.28992606e+05]
[[  1.00000000e+00   8.56697079e-21]
 [  1.00000000e+00   2.17275179e-20]] [  1.40547147e-13   1.52919033e-13] [  1.32493562e-13   9.40073861e+05]
[[  1.00

 [  1.00000000e+00   2.17275192e-20]] [  1.45238296e-13   1.58175997e-13] [  1.36816387e-13   9.83067133e+05]
[[  1.00000000e+00   8.56697267e-21]
 [  1.00000000e+00   2.17275196e-20]] [  1.47331029e-13   1.60403161e-13] [  1.38821612e-13   9.93281782e+05]
[[  1.00000000e+00   8.56696843e-21]
 [  1.00000000e+00   2.17275158e-20]] [  1.31970188e-13   1.41351007e-13] [  1.25863666e-13   7.12798499e+05]
[[  1.00000000e+00   8.56696890e-21]
 [  1.00000000e+00   2.17275162e-20]] [  1.34093896e-13   1.43633400e-13] [  1.27884075e-13   7.24856182e+05]
[[  1.00000000e+00   8.56696937e-21]
 [  1.00000000e+00   2.17275166e-20]] [  1.36217051e-13   1.45915675e-13] [  1.29903650e-13   7.36946885e+05]
[[  1.00000000e+00   8.56696984e-21]
 [  1.00000000e+00   2.17275170e-20]] [  1.38338562e-13   1.48197230e-13] [  1.31920979e-13   7.49107720e+05]
[[  1.00000000e+00   8.56697031e-21]
 [  1.00000000e+00   2.17275175e-20]] [  1.40457448e-13   1.50476980e-13] [  1.33935149e-13   7.61330932e+05]
[[  1.00

 [  1.00000000e+00   2.17275192e-20]] [  1.46947832e-13   1.59508362e-13] [  1.38771447e-13   9.54407858e+05]
[[  1.00000000e+00   8.56697267e-21]
 [  1.00000000e+00   2.17275196e-20]] [  1.49008853e-13   1.61718519e-13] [  1.40735385e-13   9.65739964e+05]
[[  1.00000000e+00   8.56696843e-21]
 [  1.00000000e+00   2.17275158e-20]] [  1.26670546e-13   1.39940633e-13] [  1.18032271e-13   1.00832334e+06]
[[  1.00000000e+00   8.56696890e-21]
 [  1.00000000e+00   2.17275162e-20]] [  1.28710332e-13   1.42147457e-13] [  1.19963323e-13   1.02101567e+06]
[[  1.00000000e+00   8.56696937e-21]
 [  1.00000000e+00   2.17275166e-20]] [  1.30749378e-13   1.44350301e-13] [  1.21895742e-13   1.03346185e+06]
[[  1.00000000e+00   8.56696984e-21]
 [  1.00000000e+00   2.17275170e-20]] [  1.32787359e-13   1.46548682e-13] [  1.23829309e-13   1.04564974e+06]
[[  1.00000000e+00   8.56697031e-21]
 [  1.00000000e+00   2.17275175e-20]] [  1.34823862e-13   1.48742200e-13] [  1.25763602e-13   1.05758050e+06]
[[  1.00

In [161]:
header = {'name':'JPA Ic6.9uA res', 'type':'Noise & Gain', 'params':['Pump power', 'Probe frequency']}
save_pkl(header, {'Gain':(noises[1]['Power'][0], noises[1]['Power'][1], gains, noises[1]['Power'][3]),
          'Noise Temp':(noises[1]['Power'][0], noises[1]['Power'][1], noise_Ts, {'log':False})})

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [166]:
axes = plt.gca()
image = axes.get_images()[0]
image.set_clim(0, 2)

In [90]:
np.linalg.lstsq([[1, 1e-20], [1, 2e-20]], [1e-10, 1.1e-10])

(array([  1.05000000e-10,   1.57500000e-30]),
 array([], dtype=float64),
 1,
 array([  1.41421356e+00,   7.07106781e-21]))

In [ ]:
nois

In [57]:
from scipy.signal import medfilt
plt.figure(figsize=(12,8))
cross_sections = [130, 140, 150, 160]
for cs in cross_sections:
    noise_Ts_filtered = noise_Ts[2][cs,:]
    noise_Ts_filtered[noise_Ts_filtered<0] = np.nan
    noise_Ts_filtered[noise_Ts_filtered>6] = np.nan
    noise_Ts_filtered_freqs = noise_Ts[1][1][np.isfinite(noise_Ts_filtered)]
    plt.scatter(noise_Ts_filtered_freqs, medfilt(noise_Ts_filtered[np.isfinite(noise_Ts_filtered)], 9), s=4, label='{0} dBm'.format(noise_Ts[1][0][cs]))
    
plt.title('Added noise temperature')
plt.xlabel('Frequency, Hz')
plt.ylabel('Temperature, K')
plt.grid(True)
plt.legend()

c:\python36\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  
c:\python36\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in greater
  import sys


In [63]:
from scipy.signal import medfilt
plt.figure(figsize=(12,8))
cross_sections = [130, 140, 150, 160]
for cs in cross_sections:
    gains_filtered = gains[2][cs,:]
    gains_filtered[gains_filtered<0] = np.nan
    #gains_filtered[noise_Ts_filtered>6] = np.nan
    gains_filtered_freqs = noise_Ts[1][1][np.isfinite(gains_filtered)]
    plt.scatter(gains_filtered_freqs, 10*np.log10(gains_filtered[np.isfinite(gains_filtered)]), s=4, label='{0} dBm'.format(noise_Ts[1][0][cs]))
    
plt.title('Gain')
plt.xlabel('Frequency, Hz')
plt.ylabel('Gain, dB')
plt.grid(True)
plt.legend()

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(noises[1]['Power'][1][1], np.log10(noises[1]['Power'][2][150,:])*10, label='40mK')
plt.plot(noises[2]['Power'][1][1], np.log10(noises[2]['Power'][2][150,:])*10, label='3.42K (10 dB)')
plt.title('Noise Power (pump power {0} dBm)'.format(noises[1]['Power'][1][0][150]))
plt.xlabel('Frequency, Hz')
plt.ylabel('Power, dBm')

In [32]:
class paramp:
    def __init__(self, vna, pump_src, bias_src):
        self.vna = vna
        self.pump_src = pump_src
        self.bias_src = bias_src
        self.optimal_points = {}
        
        self.target_f = None
        self.target_bw = None
        self.target_power = None
        
        self.hint_fp_offset = 100e6
        self.hint_rel_errors = (1e-4, 1e-2, 1e-2)
        self.hint_maxfun = 200
        self.hint_nop = 5000
        self.hint_span_bw = 30.
        
        self.pump_on = True

    def measure(self):
    # maximize SNR of S21 at given frequency, excitation power and bandwidth
        pna.set_nop(self.hint_nop)
        pna.set_bandwidth(self.target_bw)
        pna.set_power(self.target_power)
        pna.set_centerfreq(self.target_f)
        pna.set_span(self.target_bw/self.hint_span_bw)
    
        params = ((self.pump_src.set_frequency, self.pump_src.get_frequency()),
                  (self.pump_src.set_power, self.pump_src.get_power()),
                  (self.bias_src.set_current, self.bias_src.get_current()))
        initial_simplex=[[p[1]*(1+self.hint_rel_errors[p_id]) if v==p_id else p[1] for p_id,p in enumerate(params)] for v in range(len(params)+1)]
        self.pump_src.set_status(self.pump_on)
        self.bias_src.set_status(True)
    
        def target():
            time.sleep(0.4)
            measurements = pna.measure()['S-parameter'].ravel()
            print (np.log10(np.abs(np.mean(measurements)))*20, np.log10(np.std(measurements)/np.abs(np.mean(measurements)))*10)
            return np.std(measurements)/np.abs(np.mean(measurements))
    
        res = sweep.optimize(target, *params, initial_simplex = initial_simplex, maxfun=self.hint_maxfun)
        for x, p in zip(res[0], params): p[0](x)
        S21 = pna.measure()['S-parameter'].ravel()[0]
        measurement = {'S-parameter':np.asarray(S21), 
                       'SNR':np.asarray(res[1]), 
                       'Pump frequency':np.asarray(res[0][0]), 
                       'Pump power': np.asarray(res[0][1]), 
                       'Bias': np.asarray(res[0][2])}
        return measurement
    
    def set_target_f(self, value):
        self.target_f = value
    
    def set_target_bw(self, value):
        self.target_bw = value
        
    def set_target_power(self, value):
        self.target_power = value
    
    def get_opts(self):
        return {'S-parameter':{'log':20}, 
                'SNR':{'log':20}, 
                'Pump frequency':{'log':False}, 
                'Pump power':{'log':False}, 
                'Bias':{'log':False}}
    
    def get_points(self):
        return {'S-parameter':[],
                'SNR':[], 
                'Pump frequency':[], 
                'Pump power':[], 
                'Bias':[]}
    
    def get_dtype(self):
        return {'S-parameter':np.complex,
                'SNR':np.float, 
                'Pump frequency':np.float, 
                'Pump power':np.float, 
                'Bias':np.float}
                  
    def set_parameters(self, f, power, bias):
        import time
        params = self.measure()
        pump_src.set_frequency(params['Pump frequency'])
        pump_src.set_power(params['Pump power'])
        bias_src.set_current(params['Bias'])
        
    def load_calibration(self, path, name):
        import pickle
        from scipy.interpolate import interp1d
        bias_name = 'Bias Paramp VNA calib'
        pumpfreq_name = 'Pump frequency Paramp VNA calib'
        pumppower_name = 'Pump power Paramp VNA calib'
        self.calib_bias = pickle.load(open('{0}\\{1} {2}.pkl'.format(path, bias_name, name), 'rb'))[1]['Bias'][2]
        self.calib_pumpfreq = pickle.load(open('{0}\\{1} {2}.pkl'.format(path, pumpfreq_name, name), 'rb'))[1]['Pump frequency'][2]
        self.calib_pumppower = pickle.load(open('{0}\\{1} {2}.pkl'.format(path, pumppower_name, name), 'rb'))[1]['Pump power'][2]
        self.calib_targetfreq = pickle.load(open('{0}\\{1} {2}.pkl'.format(path, pumppower_name, name), 'rb'))[1]['Pump power'][1][0]
        self.pump_frequency_by_target_frequency = interp1d(self.calib_targetfreq, self.calib_pumpfreq)
        self.pump_power_by_target_frequency = interp1d(self.calib_targetfreq, self.calib_pumppower)
        self.bias_by_target_frequency = interp1d(self.calib_targetfreq, self.calib_bias)
    
    def load_noise_measurement(self, temp_files, bw):
        import pickle
        self.GN_P = []
        for T, f in temp_files:
            noise_powers = pickle.load(open(f, 'rb'))[1]['Power']
            self.GN_target_frequencies = noise_powers[1][0]
            self.GN_frequencies = noise_powers[1][1]
            self.GN_P.append(10**(noise_powers[2]/10))
        self.G = np.zeros_like(self.GN_P[0])
        self.TN = np.zeros_like(self.GN_P[0])
        self.G_1d = np.zeros(self.GN_target_frequencies.shape[0])
        self.TN_1d = np.zeros(self.GN_target_frequencies.shape[0])
        for tf_id, tf in enumerate(self.GN_target_frequencies):
            gain, noise_T = self.gain_noise(self.GN_frequencies, np.asarray(self.GN_P)[:, tf_id,:], temp_files[0][0], temp_files[1][0], bw)
            self.G[tf_id, :] = gain
            self.TN[tf_id, :] = noise_T
            self.G_1d[tf_id] = np.interp(tf, self.GN_frequencies, gain)
            self.TN_1d[tf_id] = np.interp(tf, self.GN_frequencies, noise_T)
            #print (noise_powers)
            
        self.GN_meas = {'Gain frequency dependence': (('Target frequency', 'Probe frequency'), 
                         (self.GN_target_frequencies, self.GN_frequencies),
                         self.G, {'log':10}),
                        'Noise T frequency dependence': (('Target frequency', 'Probe frequency'), 
                         (self.GN_target_frequencies, self.GN_frequencies),
                         self.TN),
                        'Gain': (('Frequency',), (self.GN_target_frequencies,), self.G_1d, {'log':10}),
                        'Noise T': (('Frequency',), (self.GN_target_frequencies,), self.TN_1d),}
        self.off_gain = np.nanmedian(np.nanmedian(self.G))
        return self.GN_meas
        
    def load_vna_noise_measurement(self, filename, vna_power, bw, atten):
        from scipy.constants import Boltzmann
        noise_vna = open('D:\\qtlab\\ReiData\\data\\2017-06-23\\10-52-50\\S-parameter S-paramter with std -60 dBm, bw 5e4.pkl', 'rb')
        data_vna = pickle.load(noise_vna)
        noise_vna.close()
        noise_rel = (np.real(data_vna[1]['S-parameter std'][2])/np.abs(data_vna[1]['S-parameter'][2]))**2
        noise_vna_inp = 1e-3*(10**(vna_power/10.))
        noise_T = noise_rel*noise_vna_inp*(10**(atten/10))/(bw*Boltzmann)
        gain = (np.abs(data_vna[1]['S-parameter'][2])**2)*10**(-atten/10)
        self.TN_m = noise_T
        self.G_m = gain
        self.GN_m_target_frequencies = data_vna[1]['S-parameter'][1][0]
        self.GN_m_probe_frequencies = data_vna[1]['S-parameter'][1][1]
        self.G_m_1d = np.zeros(self.GN_m_target_frequencies.shape[0])
        self.TN_m_1d = np.zeros(self.GN_m_target_frequencies.shape[0])
        for tf_id, tf in enumerate(self.GN_m_target_frequencies):
            self.G_m_1d[tf_id] = np.interp(tf, self.GN_m_probe_frequencies, gain[tf_id,:])
            self.TN_m_1d[tf_id] = np.interp(tf, self.GN_m_probe_frequencies, noise_T[tf_id,:])
        
        self.off_gain_m = np.nanmedian(self.G_m)
        
        self.GN_m_meas = {'Monochromatic gain frequency dependence': (('Target frequency', 'Probe frequency'), 
                         (self.GN_m_target_frequencies, self.GN_m_probe_frequencies),
                         self.G_m, {'log':10}),
                        'Monochromatic noise T frequency dependence': (('Target frequency', 'Probe frequency'), 
                         (self.GN_m_target_frequencies, self.GN_m_probe_frequencies),
                         self.TN_m),
                        'Monochromatic gain': (('Frequency',), (self.GN_m_target_frequencies,), self.G_m_1d, {'log':10}),
                        'Monochromatic noise T': (('Frequency',), (self.GN_m_target_frequencies,), self.TN_m_1d),}
        self.off_gain = np.nanmedian(np.nanmedian(self.G))
        return self.GN_meas
            
    def load_gain_saturation_measurement(self, filename, filter_kernel = (3,3,3)):
        import pickle
        from scipy.signal import medfilt, convolve
        file = open(filename, 'rb')
        data = pickle.load(file)
        data_power = np.abs(data[1]['S-parameter'][2])**2
        filter_kernel = np.ones(filter_kernel)
        data_power_filt = np.transpose(convolve(data_power, filter_kernel/np.sum(filter_kernel), mode='same'), axes=(1,0,2))
        file.close()
        data_filt_diff = np.log10(data_power_filt/data_power_filt[0,:,:])*10
        data_filt_uncompressed = (data_filt_diff>-1)
        pp, tf, pf = np.meshgrid(data[1]['S-parameter'][1][1], 
                                 data[1]['S-parameter'][1][0], 
                                 data[1]['S-parameter'][1][2], 
                                 indexing='ij')
        pp[data_filt_uncompressed]=np.nan
        pp_compression = np.nanmin(pp, axis=0)
        
        compression_1db_1d = np.zeros(data[1]['S-parameter'][1][0].shape, dtype=float)
        for tf_id, _tf in enumerate(data[1]['S-parameter'][1][0]):
            compression_1db_1d[tf_id] = np.interp(_tf, data[1]['S-parameter'][1][2], pp_compression[tf_id,:])
        
        self.sat_1db_ft_freq = pp_compression
        self.sat_1db_ft = compression_1db_1d
        
        self.sat_target_freq = data[1]['S-parameter'][1][0]
        self.sat_freq = data[1]['S-parameter'][1][2]
        
        self.sat_meas = data[1]
        self.sat_meas['1 dB compression point on frequency'] = (('Target frequency', 'Probe frequency'), \
                                                       (self.sat_target_freq, self.sat_freq), self.sat_1db_ft_freq)
        self.sat_meas['1 dB compression point'] = (('Frequency',), (self.sat_target_freq,), self.sat_1db_ft)
        
        return self.sat_meas
    
    def save_gain_saturation_plots(self, name):
        import save_pkl
        calibration_path = get_config().get('datadir')+'/calibrations/paramp/saturation/{0}/'.format(name)
        header = {'name':name, 'type':'gain saturation'}
        save_pkl.save_pkl(header, self.sat_meas, location=calibration_path)
        
    def save_gain_noise_plots(self, name):
        import save_pkl
        calibration_path = get_config().get('datadir')+'/calibrations/paramp/gain & noise/{0}/'.format(name)
        header = {'name':name, 'type':'gain & noise'}
        save_pkl.save_pkl(header, self.GN_meas, location=calibration_path)

    def save_vna_gain_noise_plots(self, name):
        import save_pkl
        calibration_path = get_config().get('datadir')+'/calibrations/paramp/monochromatic gain & noise/{0}/'.format(name)
        header = {'name':name, 'type':'Monochromatic gain & noise'}
        save_pkl.save_pkl(header, self.GN_m_meas, location=calibration_path)
            
    def planck_function(self, f, Ts, gains):
        from scipy.constants import Planck, Boltzmann
        return np.sum([Planck*f*(0.5+1./(np.exp(Planck*f/(Boltzmann*T))-1))*gain for T, gain in zip(Ts, gains)])

    def gain_noise(self, f, P_meas, T1, T2, bw):
        from scipy.constants import Boltzmann
        G_ = np.zeros_like(f)
        TN_ = np.zeros_like(f)
        P_meas = np.asarray(P_meas)*1e-3
        for f_id, f_ in enumerate(f):
            P_in = [self.planck_function(f_, 
                                         [t[0] for t in T1], 
                                         [t[1] for t in T1]), 
                    self.planck_function(f_, 
                                         [t[0] for t in T2], 
                                         [t[1] for t in T2])] # input noise powers
            a = np.asarray([[1, P_in[0]*bw], [1, P_in[1]*bw]])
            b = P_meas[:, f_id].T
            #print (f_id, a.shape, b.shape)
            solution = np.linalg.solve(a, b)
            #print (solution)
            #if (f_id<10):
                #print (a,b, solution)
            GkTNbw, G = solution
            TN = GkTNbw/(Boltzmann*G*bw)
            G_[f_id] = G
            TN_[f_id] = TN
        return G_, TN_
    
    def set_target_freq_calib(self, f):
        self.pump_src.set_frequency(self.pump_frequency_by_target_frequency(f))
        self.pump_src.set_power(self.pump_power_by_target_frequency(f))
        self.bias_src.set_current(self.bias_by_target_frequency(f))
        
    def calibrate_vna(self, frequencies, name):
        calibration_path = get_config().get('datadir')+'/calibrations/paramp/calibration'
        calibration = sweep.sweep(self, 
                                  (freqs, paramp.set_target_f, 'Target frequency'), 
                                  filename='Paramp VNA calibration {0}'.format(name), 
                                  output=False, 
                                  location=calibration_path)
        self.load_calibration(calibration_path, name)

In [167]:
#paramp = paramp(pna, lo1, current_src)
paramp = paramp(None, None, None)
paramp.set_target_f(5e9)
paramp.set_target_bw(600e3)
paramp.set_target_power(-40)
paramp.hint_maxfun = 50
paramp.pump_on = True

meas_noise = paramp.load_noise_measurement(((((3.41, 0.1), (0.040, 0.9)), 'D:\\qtlab\\ReiData\\data\\2017-06-21\\20-52-53\\Power Noise 341mK.pkl'),
                               (((0.040, 1.0),), 'D:\\qtlab\\ReiData\\data\\2017-06-21\\23-24-58\\Power Noise 40mK.pkl')), bw=3e5)
meas_saturation = paramp.load_gain_saturation_measurement('D:\\qtlab\\ReiData\\data\\2017-06-22\\10-12-10\\S-parameter VNA Gain Saturation.pkl')

file = open('D:\\qtlab\\ReiData\\data\\2017-06-22\\17-21-47\\Power VNA -70 dBm power on SA, bw 5e4.pkl', 'rb')
data = pickle.load(file)
file.close()
paramp.attenuation = np.nanmedian(data[1]['Power'][2])-np.log10(np.nanmedian(paramp.G))*10+70
print (paramp.attenuation)

meas_noise_m = paramp.load_vna_noise_measurement(\
                filename='D:\\qtlab\\ReiData\\data\\2017-06-23\\10-52-50\\S-parameter S-paramter with std -60 dBm, bw 5e4.pkl', \
                bw=5e4, atten=paramp.attenuation, vna_power=-60)

c:\python36\lib\site-packages\ipykernel_launcher.py:241: RuntimeWarning: divide by zero encountered in double_scalars


-70.3352116661


c:\python36\lib\site-packages\ipykernel_launcher.py:181: RuntimeWarning: All-NaN axis encountered


In [11]:
paramp0 = paramp(pna, lo1, current_src)
#paramp = paramp(None, None, None)
paramp0.set_target_f(5e9)
paramp0.set_target_bw(600e3)
paramp0.set_target_power(-35)
paramp0.hint_maxfun = 50
paramp0.pump_on = True


In [168]:
paramp.save_gain_saturation_plots('Kulesh 3SQ 4wave')
paramp.save_gain_noise_plots('Kulesh 3SQ 4wave')
paramp.save_vna_gain_noise_plots('Kulesh 3SQ 4wave')

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)
D:\qtlab_replacement\scripts\plotting.py:41: RuntimeWarning: divide by zero encountered in log10
  elif log == 10:									filter_abs = lambda x: np.log10(np.abs(x))*10


In [47]:
plt.figure()
plt.pcolormesh(np.log10(paramp.G)*10, vmin=40, vmax=80)
plt.colorbar()
plt.figure()
plt.pcolormesh(paramp.TN, vmin=0, vmax=10)
plt.colorbar()

c:\python36\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log10
  
c:\python36\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log10
  


c:\python36\lib\site-packages\matplotlib\colors.py:494: RuntimeWarning: invalid value encountered in less
  cbook._putmask(xa, xa < 0.0, -1)


510046.62923161994

In [54]:
np.log10(np.nanmean(paramp.G[np.isfinite(paramp.TN)]))*10

43.717889941409204

In [102]:
paramp.Gn = paramp.G.copy()
paramp.Gn[paramp.Gn<0]=0
paramp.TNn = paramp.TN.copy()
paramp.TNn[paramp.TNn<0]=0
paramp.TNn[np.isinf(paramp.TNn)]=0

In [112]:
plt.figure()
plt.pcolormesh(np.abs(data_filt_diff[:,0,:]))
plt.colorbar()

c:\python36\lib\site-packages\matplotlib\colors.py:494: RuntimeWarning: invalid value encountered in less
  cbook._putmask(xa, xa < 0.0, -1)


In [12]:
freqs = np.linspace(5.0e9, 6.0e9, 201)
lo1.set_power(12)
lo1.set_frequency(10e9)
current_src.set_current(690e-6)

True

In [14]:
sweep.sweep(paramp0, (freqs, paramp0.set_target_f, 'Target frequency'), filename='Paramp VNA calib', output=False)

Started at:  Jan 17 2018 12:59:18


root: WARNING  Setting a boolean, but that is not the expected type


[9996677590.553, 12.18, 0.000701]
-17.3950228526 -10.1572326352
[  9.99767726e+09   1.21800000e+01   7.01000000e-04] 0.0964443380152
-17.128087535 -10.0758314989
[  9.99667759e+09   1.23018000e+01   7.01000000e-04] 0.0982690709328
-17.9017514607 -9.94807776034
[  9.99667759e+09   1.21800000e+01   7.08010000e-04] 0.101202729061
-17.337835256 -10.2180360302
[  9.99667759e+09   1.21800000e+01   7.01000000e-04] 0.0951034774102
-17.5486519335 -8.95602096296
[  9.99734404e+09   1.22612000e+01   6.93990000e-04] 0.127173874772
-17.494727728 -10.1345446281
[  9.99684420e+09   1.22003000e+01   7.04505000e-04] 0.0969494918671
-17.7333429667 -9.98906793382
[  9.99745511e+09   1.20717333e+01   7.03336667e-04] 0.100252037207
-17.2053101202 -10.1810911246
[  9.99687197e+09   1.22442833e+01   7.01584167e-04] 0.0959159621283
-17.2371857806 -10.0488848554
[  9.99730701e+09   1.22025556e+01   6.97884444e-04] 0.0988806959544
-17.3670736229 -10.1153998798
[  9.99695990e+09   1.22008639e+01   7.02849861e-04

root: WARNING  Setting a boolean, but that is not the expected type


[9996760896.2, 12.19, 0.000702]
-17.3126015277 -10.0978191184
[  9.99776057e+09   1.21900000e+01   7.02000000e-04] 0.0977728080024
-17.030402921 -10.0819685769
[  9.99676090e+09   1.23119000e+01   7.02000000e-04] 0.0981303035597
-17.8634567418 -9.96712948623
[  9.99676090e+09   1.21900000e+01   7.09020000e-04] 0.100759743063
-17.2628517881 -10.2116666244
[  9.99676090e+09   1.21900000e+01   7.02000000e-04] 0.0952430594404
-17.3374253899 -9.03804184257
[  9.99742735e+09   1.22712667e+01   6.94980000e-04] 0.124794606434
-17.461613852 -10.1419663465
[  9.99692751e+09   1.22103167e+01   7.05510000e-04] 0.096783955026
-17.7031273856 -10.0125429032
[  9.99753842e+09   1.20816444e+01   7.04340000e-04] 0.0997116056385
-17.1584784386 -10.1259518898
[  9.99695528e+09   1.22543361e+01   7.02585000e-04] 0.0971415012785
-17.2330461687 -10.2273746059
[  9.99600188e+09   1.22464352e+01   7.04730000e-04] 0.094899197393
-17.317317075 -8.80127850192
[  9.99512254e+09   1.22746528e+01   7.06095000e-04] 0

...

root: WARNING  Setting a boolean, but that is not the expected type


[10046881518.162, 17.97, 0.000587]
-18.3322623739 -9.69854821763
[  1.00478862e+10   1.79700000e+01   5.87000000e-04] 0.107187755821
Trying to set too large value: 18.1497
-18.3677482767 -9.77236774876
[  1.00468815e+10   1.81497000e+01   5.87000000e-04] 0.105381220734
-18.3949316586 -9.75664589168
[  1.00468815e+10   1.79700000e+01   5.92870000e-04] 0.10576340173
-18.3867653886 -9.75936798685
[  1.00468815e+10   1.79700000e+01   5.87000000e-04] 0.105697131526
Trying to set too large value: 18.089799999999993
-18.5577784158 -9.71584310665
[  1.00458768e+10   1.80898000e+01   5.90913333e-04] 0.106761751318
Trying to set too large value: 18.059849999999997
-18.4429535928 -9.74342402946
[  1.00463792e+10   1.80598500e+01   5.89935000e-04] 0.106085883365
-18.3058537566 -9.8294796673
[  1.00473839e+10   1.79999500e+01   5.87978333e-04] 0.10400447672
-18.3441415506 -9.78767890602
[  1.00478862e+10   1.79700000e+01   5.87000000e-04] 0.105010350825
Trying to set too large value: 18.10976666666

root: WARNING  Setting a boolean, but that is not the expected type


[10047404211.824, 17.99, 0.000588]
-18.5629832773 -9.63210715229
[  1.00484090e+10   1.79900000e+01   5.88000000e-04] 0.108840188377
Trying to set too large value: 18.1699
-18.6365004409 -9.56433588435
[  1.00474042e+10   1.81699000e+01   5.88000000e-04] 0.11055195106
-18.6960059726 -9.60608268125
[  1.00474042e+10   1.79900000e+01   5.93880000e-04] 0.109494355566
-18.6137101886 -9.60937071097
[  1.00474042e+10   1.79900000e+01   5.88000000e-04] 0.109411489111
-19.2081402301 -9.38566530726
[  1.00480740e+10   1.78101000e+01   5.91920000e-04] 0.115194957609
Trying to set too large value: 18.079949999999997
-19.2175477819 -9.39324330777
[  1.00475717e+10   1.80799500e+01   5.88980000e-04] 0.114994129295
-18.5579910279 -9.62506844453
[  1.00479066e+10   1.79900000e+01   5.88000000e-04] 0.109016731128
-18.5806488578 -9.6355094256
[  1.00479066e+10   1.79900000e+01   5.90940000e-04] 0.108754956104
Trying to set too large value: 18.079949999999997
-18.5640182724 -9.64568359425
[  1.00479066e

{'Bias': (('Target frequency',),
  (array([  5.00000000e+09,   5.00500000e+09,   5.01000000e+09,
            5.01500000e+09,   5.02000000e+09,   5.02500000e+09,
            5.03000000e+09,   5.03500000e+09,   5.04000000e+09,
            5.04500000e+09,   5.05000000e+09,   5.05500000e+09,
            5.06000000e+09,   5.06500000e+09,   5.07000000e+09,
            5.07500000e+09,   5.08000000e+09,   5.08500000e+09,
            5.09000000e+09,   5.09500000e+09,   5.10000000e+09,
            5.10500000e+09,   5.11000000e+09,   5.11500000e+09,
            5.12000000e+09,   5.12500000e+09,   5.13000000e+09,
            5.13500000e+09,   5.14000000e+09,   5.14500000e+09,
            5.15000000e+09,   5.15500000e+09,   5.16000000e+09,
            5.16500000e+09,   5.17000000e+09,   5.17500000e+09,
            5.18000000e+09,   5.18500000e+09,   5.19000000e+09,
            5.19500000e+09,   5.20000000e+09,   5.20500000e+09,
            5.21000000e+09,   5.21500000e+09,   5.22000000e+09,
       

In [52]:
lo1.set_status(0)
lo1.set_frequency(9.5e9)
lo1.set_power(18)
pna.set_nop(1001)
measurement = pna.measure()
plt.figure('Gain')

pna.set_xlim(3e9, 5e9)
plt.plot(pna.get_points()['S-parameter'][0][1], np.log10(np.abs(measurement['S-parameter']))*20)

In [19]:
pna.get_points()['S-parameter']

{'S-parameter': [('Frequency',
   array([  4.00000000e+09,   4.00004001e+09,   4.00008002e+09, ...,
            4.19991998e+09,   4.19995999e+09,   4.20000000e+09]),
   'Hz')]}

## load optimized calibration of drive power&freq and bias

In [59]:
import pickle
savepath='D:\\qtlab\\ReiData\\data\\2017-11-16\\22-41-06\\'
name=''
#paramp = paramp(pna, lo1, current_src)
paramp0.load_calibration(savepath, name )

In [23]:
paramp.set_target_freq_calib(7.80e9)

In [26]:
sa.set_xlim(7.5e9, 8.3e9)
sa.set_nop(3001)
sa.set_res_bw(3e5)
sa.set_video_bw(1e3)
sweep.sweep(sa, (freqs, paramp.set_target_freq_calib, 'Target frequency'), filename='Noise 40mK')

Started at:  Jun 21 2017 23:24:58
First sweep...

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Target frequency: 7.9e+09,	average cycle time:  5.8s	8s			
Elapsed time: 0 h 11 m 45.28 s


TypeError: Could not convert None (type <class 'NoneType'>) to array

In [112]:
pna.set_xlim(7.5e9, 8.3e9)
pna.set_nop(1001)
pna.set_bandwidth(1e3)
probe_powers = np.linspace(-75, -45, 31)
sweep.sweep(pna, (freqs, paramp.set_target_freq_calib, 'Target frequency'), 
            (probe_powers, pna.set_power, 'Probe power'), 
            filename='VNA Gain Saturation')

Started at:  Jun 22 2017 10:12:10
First sweep...

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Target frequency: 7.9e+09,	Probe power:    -45,	average cycle time:  3.6s			s		
Elapsed time: 3 h 45 m 25.69 s


{'S-parameter': (('Target frequency', 'Probe power', 'Frequency'),
  (array([  7.56000000e+09,   7.56283333e+09,   7.56566667e+09,
            7.56850000e+09,   7.57133333e+09,   7.57416667e+09,
            7.57700000e+09,   7.57983333e+09,   7.58266667e+09,
            7.58550000e+09,   7.58833333e+09,   7.59116667e+09,
            7.59400000e+09,   7.59683333e+09,   7.59966667e+09,
            7.60250000e+09,   7.60533333e+09,   7.60816667e+09,
            7.61100000e+09,   7.61383333e+09,   7.61666667e+09,
            7.61950000e+09,   7.62233333e+09,   7.62516667e+09,
            7.62800000e+09,   7.63083333e+09,   7.63366667e+09,
            7.63650000e+09,   7.63933333e+09,   7.64216667e+09,
            7.64500000e+09,   7.64783333e+09,   7.65066667e+09,
            7.65350000e+09,   7.65633333e+09,   7.65916667e+09,
            7.66200000e+09,   7.66483333e+09,   7.66766667e+09,
            7.67050000e+09,   7.67333333e+09,   7.67616667e+09,
            7.67900000e+09,   7.68183

In [ ]:
sa.set_span(0)
sa.set_nop(1)
sa.set_res_bw(5e4)
sa.set_video_bw(1e2)
pna.set_power(-70)
pna.set_nop(1)
probe_freqs = np.linspace(7.5e9, 8.3e9, 401)
sweep.sweep(sa, (freqs, paramp.set_target_freq_calib, 'Target frequency'), 
            (probe_freqs, lambda x: (pna.set_xlim(x, x), sa.set_xlim(x,x)), 'Probe frequency'), 
            filename='VNA -70 dBm power on SA, bw 5e4', output=False)

Started at:  Jun 22 2017 17:21:47


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 4 h 19 m 39.72 s,	parameter values: Target frequency: 7.668e+09,	Probe frequency: 7.68e+09,	average cycle time: 0.47s		

In [60]:
pna.set_power(-40)
pna.set_nop(101)
pna.set_span(1e3)
pna.set_bandwidth(5e2)

vna_std = data_reduce.data_reduce(pna)
vna_std.filters = {'S-parameter': data_reduce.mean_reducer(pna, 'S-parameter', 0),
                   'S-parameter std': data_reduce.std_reducer(pna, 'S-parameter', 0)}

freqs = np.linspace(4.53e9, 4.73e9, 201)
sweep.sweep(vna_std, (freqs, paramp0.set_target_freq_calib, 'Target frequency'), 
            (freqs, pna.set_centerfreq, 'Probe frequency'), 
            filename='S-paramter with std -40 dBm, bw 5e2', output=False)

Started at:  Nov 17 2017 08:14:49


KeyboardInterrupt: 

In [ ]:
std_reduce = 

In [11]:
# four-frequency lower sideband
lo1.set_frequency(7.786e9)
lo1.set_power(13.45)
current_src.set_current(86.60e-6)
# lower f: 7.560e9, upper: 7.922e9

True

In [38]:
#three-frequency
current_src.set_current(80.43e-6)
lo1.set_frequency(16.351e9)
lo1.set_power(0.48)

True

In [10]:
current_src.set_current(84.01e-6)
lo1.set_frequency(7.92e9)
lo1.set_power(4.7)

True

In [39]:
# four-wave mixing, upper sideband
current_src.set_current(82.01e-6)
lo1.set_frequency(7.9942e9)
lo1.set_power(-0.44)
#8.081GHz

True

In [29]:
lo1.set_status(True)
pna.set_xlim(7.8e9, 8.2e9)
pna.set_nop(1001)
pna.set_bandwidth(1e5)
plt.plot(pna.get_freqpoints(), np.log10(pna.get_tracedata()[0])*20)

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [30]:
current_src.set_current(-62.74e-6)
lo1.set_frequency(8.6432e9)
lo1.set_power(-5)
target_freqs = np.arange(8.081e9, 7.800e9, 1e6)
optim_params = [paramp_optimize(pna, f, 1e3, -45, lo1, current_src, fp_guess=f+0.2e6, initial_simplex=simplex, maxfun=40) for f in target_freqs]

NameError: name 'paramp_optimize' is not defined

In [10]:
m = sweep.sweep(sa, (pump_powers, lo1.set_power, 'Excitation power'), \
                        filename='WTJPA Ic6.9uA res pump{0:2.4f}GHz switch 1'.format(pump_freq/1e9))

NameError: name 'pump_powers' is not defined

In [10]:
rf_switch.do_set_switch(2, channel=sw_ch)

In [19]:
plt.figure()
plt.pcolormesh(T1_noise['Power'][1][1], T1_noise['Power'][1][0], T1_noise['Power'][2]);
plt.colorbar()

In [20]:
plt.figure()
plt.pcolormesh(T2_noise['Power'][1][1], T2_noise['Power'][1][0],T2_noise['Power'][2]);
plt.colorbar()

In [21]:
plt.figure()
plt.pcolormesh(T2_noise['Power'][1][1], T2_noise['Power'][1][0], T1_noise['Power'][2]-T2_noise['Power'][2]);
plt.colorbar()

In [8]:
points = sa.get_points()
dtypes = sa.get_dtype()
meas = sa.measure()

In [9]:
pna_points = pna.get_points()
pna_dtypes = pna.get_dtype()
pna_meas = pna.measure()

In [10]:
print(points['Power'][0][1].shape)
print(pna_points['S-parameter'][0][1].shape)
print(dtypes)
print(pna_dtypes)
print(meas['Power'].shape)
print(pna_meas['S-parameter'].shape)

(1000,)
(201,)
{'Power': <class 'float'>}
{'S-parameter': <class 'complex'>}
(1000,)
(201,)


### Загрузка данных

In [36]:
filename = 'Noise & Gain JPA Ic6.9uA res'
location = 'D:\\qtlab\\ReiData\\data\\2017-06-15\\13-33-17'
data = load_pkl(filename, location=location)[1]

D:\qtlab\ReiData\data\2017-06-15\13-33-17/Noise & Gain JPA Ic6.9uA res.pkl


In [37]:
gains = data['Gain']
noise_Ts= data['Noise Temp']

In [199]:
data_cropped = data.copy()
data_cropped['S-parameter'] = (data_cropped['S-parameter'][0], \
                               (data_cropped['S-parameter'][1][0], data_cropped['S-parameter'][1][1][150:]), \
                               data_cropped['S-parameter'][2][:,150:])
data_cropped_amp = data.copy()
data_cropped_amp['S-parameter'] = (data_cropped['S-parameter'][0], \
                               (data_cropped['S-parameter'][1][0], data_cropped['S-parameter'][1][1]), \
                               convolve2d(data_cropped['S-parameter'][2], np.ones((4,4)), mode='same'))

In [190]:
convolve2d(data_cropped['S-parameter'][2], np.ones((1,1)), mode='same')[:,150:].shape

(401, 201)

In [234]:
import fitting
import imp
imp.reload(fitting)
from scipy.signal import convolve2d

In [235]:
results = fitting.dc_squid_fit_S21(data_cropped_amp, noise_sigma=3, fit=False, method='diff_curr', plot=True)

c:\python36\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\python36\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [228]:
results.keys()

dict_keys(['fitresults', 'evaluator', 'fp_est', 'fp_est_xpoints'])

In [237]:
results['fitresults']

{'I0': 5.1017243648743762e-05,
 'L': 4234.4902819693152,
 'a': 1.0000386770339402,
 'b': -1.0000298591138239,
 'fp': 3425223051.333847}

In [238]:
plt.figure()
plt.pcolormesh(*data_cropped['S-parameter'][1], np.abs(data_cropped['S-parameter'][2]).T)
plt.scatter(results['fp_est_xpoints'],results['fp_est'])
plt.plot(data_cropped['S-parameter'][1][0], results['evaluator'](data_cropped['S-parameter'][1][0]))
plt.xlim([np.min(data_cropped['S-parameter'][1][0]), np.max(data_cropped['S-parameter'][1][0])])
plt.ylim([np.min(data_cropped['S-parameter'][1][1]), np.max(data_cropped['S-parameter'][1][1])])

(2100000000.0, 3500000000.0)

In [15]:
S21 = data['S-parameter'][2]
currents = data['S-parameter'][1][0]
frequencies = data['S-parameter'][1][1]
fitresults = [None]*currents.size

In [5]:
from scipy.ndimage.filters import gaussian_filter
from scipy.optimize import leastsq
bgremove = np.sum(S21[0:2,:], axis=0)/3
_S21 = S21
S21 = S21/bgremove
#_S21_filt = (gaussian_filter(np.real(_S21), 2)+1j*gaussian_filter(np.imag(_S21), 2))
delay = -(np.unwrap(np.angle(bgremove))[-1] - np.unwrap(np.angle(bgremove))[0])/(2*np.pi*(frequencies[-1]-frequencies[0]))
plt.figure('S21')
diff = np.gradient((gaussian_filter(np.real(S21), 3)+1j*gaussian_filter(np.imag(S21), 3)))[1]/(frequencies[1]-frequencies[0])
diff2 = np.gradient((np.real(diff)+1j*np.imag(diff)))[1]/(frequencies[1]-frequencies[0])
plt.pcolormesh(currents, frequencies, np.abs(S21.T))#np.abs(np.gradient(S21.T)[1]))
res_freq_id_estimate = np.argmin(np.abs(S21), axis=1)
res_freq_estimate = frequencies[res_freq_id_estimate]
#delay_mesh = -np.gradient(np.unwrap(np.angle(_S21_filt)))[1]/((frequencies[1]-frequencies[0])*2*np.pi)-delay
#delay2_mesh = np.gradient(delay_mesh)[1]/((frequencies[1]-frequencies[0])*2*np.pi)
res_delay = diff[[i for i in range(currents.size)], res_freq_id_estimate]
res_delay_deriv = diff2[[i for i in range(currents.size)], res_freq_id_estimate]
lifetime = res_delay_deriv/res_delay*1j/(np.pi)
Q = (lifetime*res_freq_estimate)/2.
C = lifetime/(2*50) # in Farads
L = 1/((2*np.pi*res_freq_estimate)**2*C)
#res_freq_estimate_unreliable = np.zeros(currents.shape, dtype=bool)
res_freq_estimate_unreliable = np.min(np.abs(S21), axis=1)>np.mean(np.abs(S21))-3*np.std(np.abs(S21))
for cur_id, cur in enumerate(currents):
    if cur_id < 1 or cur_id >= currents.size-1:
        continue
    if (np.abs(res_freq_estimate[cur_id]-np.mean(res_freq_estimate[cur_id-3:cur_id+3])))>(np.max(frequencies)-np.min(frequencies))*0.1:
        res_freq_estimate_unreliable[cur_id] = True

res_freq_estimate[res_freq_estimate_unreliable] = np.nan
fit_unreliable = np.zeros(currents.shape, dtype=bool)
plt.plot(currents, res_freq_estimate)
fit_fr = np.zeros(currents.shape)
fit_Qc = np.zeros(currents.shape)
fit_Ql = np.zeros(currents.shape)
fit_Qi = np.zeros(currents.shape)
Qc_column_name = 'Qc'
Qi_column_name = 'Qi'
Ql_column_name = 'Ql'

fit = False
if fit:
    last_freq = False
    span = 80e6
    for cur_id, cur in enumerate(currents):
        fit_unreliable[cur_id] = True
        if not last_freq and np.isfinite(res_freq_estimate[cur_id]):
            f0 = res_freq_estimate[cur_id]
        elif np.isfinite(last_freq):
            f0 = last_freq
        else:
            continue
        
        if (f0-span/2.<np.min(frequencies) or f0+span/2.>np.max(frequencies)):
            last_freq = False
            continue
        
        freq_mask = (frequencies>f0-span/2.)*(frequencies<f0+span/2.)
        f_data = frequencies[freq_mask]
        z_data = S21[cur_id, :][freq_mask]
        
        fitter = circuit.reflection_port(f_data=f_data, z_data_raw=z_data)
        fitter.autofit()
        fitresults[cur_id] = fitter.fitresults
        fit_fr[cur_id] = fitresults[cur_id]['fr']
        fit_Qc[cur_id] = fitresults[cur_id][Qc_column_name]
        fit_Qi[cur_id] = fitresults[cur_id][Qi_column_name]
        fit_Ql[cur_id] = fitresults[cur_id][Ql_column_name]
        fit_unreliable[cur_id] = (1-(fit_fr[cur_id]<np.nanmax(f_data))*(fit_fr[cur_id]>np.nanmin(f_data)))>0
        
    fit_fr[fit_unreliable] = np.nan
    fit_Qi[fit_unreliable] = np.nan
    fit_Qc[fit_unreliable] = np.nan
    fit_Ql[fit_unreliable] = np.nan
    #plt.plot(currents, fit_fr)
    #plt.ylim([np.min(frequencies), np.max(frequencies)])
    #plt.figure('Qc')
    fit_Qc[fit_Qc>1e5]=np.nan
    fit_Qc[fit_Qc<0]=np.nan
    
# fitting res_freq_estimate to theoretical
ft = lambda I,fp,I0,L,a,b: np.real(fp*(1+b)*(1/(1/np.sqrt((1-a)*np.abs(np.cos((I-I0)*L).astype(np.complex))+a)+b)))
ft_x = lambda I,x: ft(I, x[0], x[1], x[2], x[3], x[4])
currents_freq_estimate = currents[np.isfinite(res_freq_estimate)]
cost = lambda x: ft_x(currents_freq_estimate, x)-res_freq_estimate[np.isfinite(res_freq_estimate)]
max_freq_cur_id = np.nanargmax(res_freq_estimate)
max_freq = np.nanmax(res_freq_estimate)
cur_h = 3
d2fdI2 = (res_freq_estimate[max_freq_cur_id+cur_h]+res_freq_estimate[max_freq_cur_id-cur_h]-2*res_freq_estimate[max_freq_cur_id])/\
            ((currents[max_freq_cur_id]-currents[max_freq_cur_id-cur_h])**2)
x0 = [max_freq, currents[np.nanargmax(res_freq_estimate)], np.sqrt(-d2fdI2/max_freq), 0.5, 0.5]
res_freq_cur_fit = leastsq(cost, x0)
plt.plot(currents, ft_x(currents, res_freq_cur_fit[0]))
plt.ylim([np.min(frequencies), np.max(frequencies)])
#plt.plot(currents, ft_x(currents, x0))
S21 = _S21

c:\python36\lib\site-packages\numpy\core\fromnumeric.py:2889: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\python36\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\python36\lib\site-packages\ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in sqrt


In [9]:
res_freq_cur_fit

(array([  5.12808991e+09,  -1.13186206e-04,   8.20829356e+03,
          1.00250435e-01,   2.31468485e+00]), 1)

In [67]:
plt.figure()
#plt.plot(currents, np.real(Q))
plt.plot(currents, np.real(lifetime))
plt.ylim([0, 20e-9])
plt.grid()
plt.ylabel('Relaxation time, s')
plt.figure()
#plt.plot(currents, np.real(Q))
plt.plot(currents, np.real(C))
plt.ylim([-0.2e-9, 0.2e-9])
plt.grid()
plt.ylabel('Full resonator capacitance, F')
plt.figure()
#Lg = (res_freq_cur_fit[0][4]+1)/res_freq_cur_fit[0][4]*np.real(L)
#Lj = (res_freq_cur_fit[0][4]+1)*np.real(L)
plt.plot(currents, np.real(L), label='Full inductance')
#plt.plot(currents, np.real(Lg), label='Geometric inductance')
#plt.plot(currents, np.real(Lj), label='Josephson inductance')
plt.ylim([-0e-9, 0.2e-10])
plt.grid()
plt.ylabel('Resonator inductance, H')

In [8]:
Q_est = float(Q[max_freq_cur_id])
C_est = float(C[max_freq_cur_id])
L_est = float(L[max_freq_cur_id])
Lj = res_freq_cur_fit[0][4]/(res_freq_cur_fit[0][4]+1)*np.real(L_est)
Lg_est = np.real(L_est)/(res_freq_cur_fit[0][4]+1)
Ic = 1.054e-34/(2*1.6e-19*Lj)
print ('fr: {6:4.3g}, C: {0:4.2g} F, Lg: {1:4.2g} Hn, Lj: {2:4.2g} Hn, full SQUID Ic: {3:4.2g} A, asymmetry: {4:4.2f}, Q: {5:4.2f}'.format(\
        C_est, Lg_est, Lj, Ic, res_freq_cur_fit[0][3], Q_est, res_freq_cur_fit[0][0]))

fr: 5.13e+09, C: 1.5e-10 F, Lg: 1.9e-12 Hn, Lj: 4.5e-12 Hn, full SQUID Ic: 7.3e-05 A, asymmetry: 0.10, Q: 38.23


c:\python36\lib\site-packages\ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.
c:\python36\lib\site-packages\ipykernel_launcher.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  
c:\python36\lib\site-packages\ipykernel_launcher.py:3: ComplexWarning: Casting complex values to real discards the imaginary part
  This is separate from the ipykernel package so we can avoid doing imports until


In [175]:
bgremove

array([  2.12221228e-03 +8.28879614e-04j,
         2.50260501e-03 -5.88461660e-04j,
         1.69399094e-03 -1.95215503e-03j,
        -2.40126198e-04 -2.81517223e-03j,
        -1.62253211e-03 -2.07307943e-03j,
        -1.92226369e-03 -6.35541743e-04j,
        -2.16575161e-03 +1.15822721e-03j,
        -1.14751056e-03 +1.07392832e-03j,
        -4.58683906e-04 +2.15715566e-03j,
         1.19493428e-03 +1.80242350e-03j,
         2.05764989e-03 +1.14208658e-03j,
         2.10304504e-03 -1.68959780e-04j,
         1.40100663e-03 -1.50440400e-03j,
         5.18023599e-04 -2.79801944e-03j,
        -1.30725427e-03 -3.09538996e-03j,
        -2.24986866e-03 -1.43606328e-03j,
        -2.35149513e-03 +7.71723086e-04j,
        -2.43764518e-03 +2.29205587e-03j,
        -9.35024795e-04 +2.72559592e-03j,
         8.24331364e-04 +2.32659808e-03j,
         2.69825912e-03 +1.75517534e-03j,
         2.84684721e-03 +3.70482613e-04j,
         2.76233489e-03 -1.44801809e-03j,
         1.11265880e-03 -2.6484020

In [ ]:
plt.figure()
plt.pcolormesh(np.abs(S21.T))
plt.figure()
plt.pcolormesh(np.angle(S21.T))

In [106]:
x0

[5096000000.0, -0.00011399999999999998, 13207.147529172158]

In [51]:
fitresults

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,